# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier
In this notebook, a model to classify traffic signs is designed. The data set for the project is based on the [German Traffic Sign Dataset](https://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset). This dataset is used to train and the test the model design.
Please refer to the [result file](./Traffic_Sign_Classifier.html) for the final results of the project.
The [project writeup](./README.md) file summarizes the project results. In addition, the writeup addresses my approach towards data analysis, .. [TODO](TODO)

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import pandas as pd
import cv2
import random
import numpy as np
import bz2
import _pickle as cPickle
from matplotlib import pyplot as plt

use_augmented_datafile = False

def load_pickle_data():
    # Load pickled data
    if use_augmented_datafile :
        training_file = '..//data/augmented_data.pbz2'
        print('Uncompressing data file. This will take a while ...')
        train = bz2.BZ2File(training_file, 'rb')
        dist_pickle = cPickle.load(train)
        print('Loading of data file is complete.')
        X_train = dist_pickle['X_train'].astype('uint8') 
        y_train = dist_pickle['y_train'].astype('uint8') 
    else:
        training_file = '..//data/train.p'
        with open(training_file, mode='rb') as f:
            train = pickle.load(f)
        X_train, y_train = train['features'], train['labels']
        
    validation_file= '..//data/valid.p'
    testing_file = '..//data/test.p'
    
    
    with open(validation_file, mode='rb') as f:
        valid = pickle.load(f)
    with open(testing_file, mode='rb') as f:
        test = pickle.load(f)
        
    
    X_valid, y_valid = valid['features'], valid['labels']
    X_test, y_test = test['features'], test['labels']

    df =  pd.read_csv('./signnames.csv') 
    label_texts = list(df['SignName']) 
    label_ids = list(df['ClassId'])
    #since label texts are already sorted in the CSV file, the signnames array
    #index can be directly used for 'mapping'
    return X_train, y_train, X_valid, y_valid, X_test, y_test, label_texts, label_ids


X_train, y_train, X_valid, y_valid, X_test, y_test, label_texts, label_ids = load_pickle_data()

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

### Basic Summary of the Data Set

In [ ]:
def summarize_data(X_train, y_train, X_valid, y_valid, X_test, y_test):
    
    # Number of training examples
    n_train = X_train.shape[0]
    
    # Number of validation examples
    n_validation = X_valid.shape[0]
    
    # Number of testing examples.
    n_test = X_test.shape[0]
    
    # The shape of an traffic sign image
    # The Input 'features' is a 4D array containing raw pixel data of the traffic sign 
    # images, (num examples, width, height, channels).
    image_shape = X_train[0].shape
    
    # unique classes/labels there are in the dataset.
    n_classes = len(np.unique(y_train))
    
    print("Number of training data =", n_train)
    print("Number of validation data =", n_validation)
    print("Number of testing data =", n_test)
    print("Image data shape =", image_shape)
    print("Number of classes =", n_classes)

    return n_train, n_validation , n_test, image_shape, n_classes

#prints out the count of data in the training, validation and test data sets
n_train, n_validation , n_test, image_shape, n_classes = summarize_data(X_train, y_train, \
                                                                        X_valid, y_valid, \
                                                                        X_test, y_test)
# check if the count of inputs match with the corresponding labels
assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test)) 

### Exploratory visualization of the dataset

In this section, we see a sample set of images from the database. Later, the distribution of classes in the training, validation and test sets are plotted. 

#### Data exploration visualization 

In [ ]:
def random_plot(X_train, y_train, label_texts, image_count):
    images_per_row = 8
    #count more than the mulitples of 'images_per_row' will be ignored
    rows = int(image_count/images_per_row)
    length_train_data = len(X_train)
    
    #prepare a random 'index' list from the training dataset
    indexs = random.sample(range(length_train_data), image_count)
    #plot the randomly selected signs 
    fig, axes = plt.subplots(rows,images_per_row, figsize=(24,12))
    for ax , index in zip( axes.flatten(), indexs) :
        ax.imshow(X_train[index].astype('uint8'))
        #picks the labels from the 'master' labels loaded from signposts.csv
        ax.set_title(label_texts[y_train[index]])
        
random_plot(X_train, y_train, label_texts, 24)

In [ ]:
### Distribution of classes (Histogram) 
def draw_horz_plot(labels, data, title):
    n_classes = len(np.unique(data))
    count_classes = np.zeros(n_classes)
    for i in range (n_classes):
        count = np.where(data == i )
        count_classes[i] = len(count[0])
    #plot a bar chart to show the frequency of the signnames in the training set
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(1,1,1)
    ax.grid(True)
    ax.set_title(title, fontsize=30)
    ax.set_xlabel('Data count', fontsize=20)
    plt.barh( labels, count_classes, height = 0.8 , align = 'center' )
    
def draw_histogram(y_train, y_valid, y_test):
    n_classes = len(np.unique(y_train))
    fig, axes = plt.subplots(ncols=3, figsize=(16,4))
    for ax in axes:
        ax.grid(True)
        ax.set_xlabel('class-id')
        ax.set_ylabel('count')
    axes[0].hist(y_train, bins=n_classes)
    axes[0].set_title('Train')
    axes[1].hist(y_valid, bins=n_classes)
    axes[1].set_title('Validation')
    axes[2].hist(y_test, bins=n_classes)
    axes[2].set_title('Test')
    fig.tight_layout()

### Plotting the distribution in the Training data set

In [ ]:
#prepare labels in the format LABEL[ID]
labels = [ label + '[' + str(id)+ ']' for label, id in zip(label_texts,label_ids)]
draw_horz_plot(labels, y_train, 'Training data')

### Plotting the distribution in the Validation data set

In [ ]:
draw_horz_plot(labels, y_valid, 'Validation data')

### Plotting the distribution in the Test data set

In [ ]:
draw_horz_plot(labels, y_test, 'Test data')

### [redundant] Plot the distribution of labels in training , validation and test datasets

In [ ]:
#Plot the distribution of labels in training , validation and test datasets
draw_histogram(y_train, y_valid, y_test)

> **Summary:**: The histogram (when generated without the augmented datasets) shows that the distribution of the classes are unbalanced. **Augumentation** techniques needs to be applied (Further details in pre-processing chapter) to those classes which are under-represented.

----

## Step 2: Design and Test a Model Architecture

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is taken as a starting point. 

Before building the model, the images in the Training and validation test should be pre-processed so that the neural network can work better.

### Pre-processing of the Data Set

### Data Augmentation
From the histogram data from the previous chapter, it is clear that the training samples are under-represented. For example, the class "Speed Limit 20km/h" is represented by 180 samples, while the class "Speed Limit 20km/h" is represented by more than 2000 samples. Augmentation of training data with pseudo images will balance out this under-representation
**NOTE** - The Data Augmentation related code will be run only if the flag is `use_augmented_datafile` is set to `True`

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

#generates pseudo images based on the training set
def augment_pseudo_data(X_train, y_train, target_class_size = 2500):
    n_classes = len(np.unique(y_train))
    datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.3,
                                         height_shift_range=0.2, shear_range=0.2, 
                                         zoom_range=0.2)
    #Initialise empty arrays to hold the pseudo images and labels
    X_pseudo = np.empty((0,32,32,3), dtype='uint8')
    y_pseudo = np.empty(0 , dtype='uint8')
    # iterate through all the labels
    for i in tqdm(range(n_classes)):
        # "Real" Images refer to those images in the German Traffic Sign Database
        # filter out the current set of "real" images and labels
        X_real = X_train[y_train == i]
        y_real = np.repeat(i, X_real.shape[0])
        # define a container for holding the pseudo images and labels
        # "Pseudo" refers to the immages and labels created by ImageDataGenerator.
        _X_pseudo = np.empty((0,32,32,3), dtype='uint8')
        _y_pseudo = np.empty(0, dtype='uint8')
        # Pass on the real images belonging to the respective class
        # keras.ImageDataGenerator does all the hard work and returns the pseudo images
        # and labels in x and y respectively
        for x,y in datagen.flow(X_real, y_real, batch_size=len(y_real), seed=i):
            #append the generated images and labels to "Pseudo" array
            _X_pseudo = np.concatenate((_X_pseudo, x), axis = 0)
            _y_pseudo = np.concatenate((_y_pseudo, y), axis = 0)
            # copy the pseudo images and labels to the master array
            if (X_real.shape[0] + _X_pseudo.shape[0] > target_class_size):
                # Break when the total (Real+Pseudo) counts of images per class reaches
                # the target limit 
                extra_images = target_class_size - _X_pseudo.shape[0] - X_real.shape[0]
                _X_pseudo = _X_pseudo[:extra_images]
                _y_pseudo = _y_pseudo[:extra_images]
                break
        # add the pseudo image for the current class
        X_pseudo = np.concatenate((X_pseudo, _X_pseudo), axis = 0)
        y_pseudo = np.concatenate((y_pseudo, _y_pseudo), axis = 0)
    #return the complete array
    return X_pseudo , y_pseudo

In [ ]:
#Call the augment function on the training data.
#if not use_augmented_datafile:
X_pseudo, y_pseudo = augment_pseudo_data(X_train, y_train, target_class_size=2500)

In [ ]:
#Call the random plot to verify the augmented data
#if not use_augmented_datafile:
random_plot(X_pseudo, y_pseudo, label_texts, image_count=16)

#### Merge pseudo data into the Training set

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

print ('Training samples before augmentation : ' + str(X_train.shape[0]))
print ('Validation samples before augmentation : ' + str(X_valid.shape[0]))

X_train = np.concatenate((X_train, X_pseudo), axis=0)
y_train = np.concatenate((y_train, y_pseudo), axis=0)

print ('Training samples after augmentation : ' + str(X_train.shape[0]))
print ('Validation samples after augmentation : ' + str(X_valid.shape[0]))

print ('Splitting Training and validation data sets after augmentation  ... ( This may take a while! )')

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

print ('Training samples after splitting : ' + str(X_train.shape[0]))
print ('Validation samples after splitting : ' + str(X_valid.shape[0]))

#### Verification of the distribution of the labels after augmentation

In [ ]:
#prepare the labels for chart
labels = [ label + '[' + str(id)+ ']' for label, id in zip(label_texts,label_ids)]
draw_horz_plot(labels, y_train, 'Training data (after Augmentation)')    

### Serializing the augmented training data set

To speed up the development process, the pesudo files along with the real training data are saved into a pickle file. The classic pickling using pickle class resulted in a pickle file which was more than 1.05 GB. With the help of the [following article](https://medium.com/better-programming/load-fast-load-big-with-compressed-pickles-5f311584507e), i could save more than 80% disk space at the cost of excuetion time.

In [ ]:
def save_augmented_data(title, X_train, y_train ):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:#
        data_pickle = {}
        print ("Saving object with {} items ".format(str(X_train.shape)))
        data_pickle['X_train'] = X_train
        data_pickle['y_train'] = y_train
        cPickle.dump(data_pickle, f)

In [ ]:
if not use_augmented_datafile:
    save_augmented_data("./augmented_data.pbz2", X_train, y_train)

### Gray scaling, Histogram equalization and Normalization

After augmentation of pseudo images, the next pre-processing steps are applied to the larger dataset

#### Basic functions for pre-processing

In [ ]:
# Normalize the pixels of the images in the given data set
def normalize(images):
    result = np.zeros((len(images), 32, 32, 3), np.float64)
    for index, image in enumerate(images):
        gray = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY), cv2.COLOR_GRAY2RGB)
        result[index] = (gray.astype(np.float64) - 128)/128
    return result

The entire training set images are normalized so that the mean and standard deviation of the **complete** training set is then used as the centering data for the validation and test data sets.

In [ ]:
X_train = normalize(X_train)
X_valid = normalize(X_valid)
X_test = normalize(X_test)

print ('Training samples after normalization : ' + str(X_train.shape[0]))
print ('Validation samples before normalization : ' + str(X_valid.shape[0]))
print ('Test samples before normalization : ' + str(X_test.shape[0]))

In [ ]:
# Visual comparison of the pre-processing steps (TODO)
plt.imshow(X_train[678])

>**Summary:** The above plot shows that the normalization process had led to a mean value closer to 0 and equal varaince across the data set

### Train, Validate and Test the Model

## Model Architecture

The model is implemented based on the [LeNet-5](http://yann.lecun.com/exdb/lenet/) neural network architecture.

#### Input
The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. During the pre-processing step, the images are converted to grayscale, which reduces the color depth to 1. Hence C is 1 in this case.

### Architecture ( TODO )
**Layer 1: Convolutional.** The output shape should be 28x28x6.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using `tf.contrib.layers.flatten`, which is already imported for you.

**Layer 3: Fully Connected.** This should have 120 outputs.

**Activation.** Your choice of activation function.

**Layer 4: Fully Connected.** This should have 84 outputs.

**Activation.** Your choice of activation function.

**Layer 5: Fully Connected (Logits).** This should have 10 outputs.

### Output
Return the result of the 2nd fully connected layer.

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten

EPOCHS = 40
BATCH_SIZE = 128

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 16), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(16))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 16, 64), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(64))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(1600, 240), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(240))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)
    
    # Dropout
    fc1 = tf.nn.dropout(fc1, keep_prob)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(240, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)

    # Dropout
    fc2 = tf.nn.dropout (fc2, keep_prob)
    
    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

## Train, Validate  and Testing the model

`x` is a placeholder for a batch of input images.
`y` is a placeholder for a batch of output labels.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)
keep_prob = tf.placeholder(tf.float32)

## Training Pipeline
Create a training pipeline that uses the model to classify Traffic sign data.

In [ ]:
rate = 0.001

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

## Model Evaluation
Evaluate how well the loss and accuracy of the model for a given dataset.

You do not need to modify this section.

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

## Train the Model
Run the training data through the training pipeline to train the model.

Before each epoch, shuffle the training set.

After each epoch, measure the loss and accuracy of the validation set.

Save the model after training.

You do not need to modify this section.

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    validation_accuracy_list = []
    print('Training...')
    print ()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.6})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        validation_accuracy_list.append (validation_accuracy)
        print('EPOCH {} ...'.format(i+1))
        print('Validation Accuracy = {:.3f} '.format(validation_accuracy))
        print()
        
    training_accuracy = evaluate (X_train, y_train)  
    print('Accuracy: Train: {:.3f}, Validation: {:.3f}'.format(training_accuracy, validation_accuracy))
    saver.save(sess, './models/traffic_sign')
    print("Model saved")

### Training key performance indicators

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(5, 4))
ax.plot(range(1, EPOCHS + 1), validation_accuracy_list)
ax.set_xlabel('Epochs')
ax.set_ylabel('Validation Accuracy')
ax.grid(True)

## Evaluate the Model
Once you are completely satisfied with your model, evaluate the performance of the model on the test set.

Be sure to only do this once!

If you were to measure the performance of your trained model on the test set, then improve your model, and then measure the performance of your model on the test set again, that would invalidate your test results. You wouldn't get a true measure of how well your model would perform against real data.

You do not need to modify this section.

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")